Veri işleme ve analizi için pandas kütüphanesi kullanılacaktır. Pandas kütüphanesi pd rumuzu ile kısaltılmıştır.

Veri işleme ve temizliği için math kütüphanesi kullanılacaktır.

In [3]:
import pandas as pd
import math

Verilerimiz Google Drive (bulut) üzerinde olduğu için Google Colab ile Google Drive arasında bağlantı kurulacaktır.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 isimli klasörde verilerimizi sakladığımız için bu klasöre erişilecektir.

***NOT:*** Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!

In [ ]:
%cd drive/
%cd MyDrive/
%cd Colab Notebooks/
%cd TDDI2022/

%ls

Veri İskeleti TDDI2022 klasöründe bulunan yarışma kapsamında yarışmacılara sağlanan **kanunum-nlp-doc-analysis-dataset.csv** dosyasının içindeki verileri pandas ile okuyarak oluşturulacaktır.

Veri iskeleti oluşturulduktan sonra veri iskeleti ile ilgili bilgi ekrana yazdırılacaktır. Tam özetin yazdırılıp yazdırılmayacağı durumunu *verbose=True* (yazdırılsın) parametresi ile kontrol edilecektir. 

In [ ]:
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv')
df.info(verbose=True)

Verilerde yapacağımız ön çalışmada tüm belgelerde mükerrer no'yu arayacak kodun çalışabilmesi için belgeler arasında ortak noktalar aramamız lazım. Bunun için belgelerden  birinde arama yapacağımız kısmı inceledik.

# ***ÇIKARIM:***
- Mükerrer No belgelerin genelikle **ilk 300 karakterinde** bulunmaktadır.
- **(Önemli Bulgu)** Mükerrer No **kanunum-nlp-doc-analysis-dataset.csv** dosyasında **"numpy.float64"** türünde işlenmiştir. Bu yüzden bizim 
çıktılarımızla uyuşması için iki değerin de **float veri tipine** çevrilmesi gereklidir.
- **(Önemli Bulgu)** **kanunum-nlp-doc-analysis-dataset.csv** dosyasında bazı satırlar için Mükerrer No işlenmemiştir ve bu satırlar çağrıldıklarına nan (Not a Number) döndürmektedir. Bunun önüne geçilmesi için math kütüphanesinde **isnan()** fonksiyounyla nan olan satırlara 0.0 ifadesi ilişkilendirilir.
- Belgelerde **Maddeler içerisinde olmamak şartıyla** X. Mükerrer ifadesi geçiyorsa Mükerrer No X.0 'dır
- Belgelerde **Maddeler içerisinde olmamak şartıyla** sadece Mükerrer ifadesi geçiyorsa Mükerrer No 1.0 'dır
- Belgelerde Mükerrer ifadesi bulunmuyorsa **Maddeler içerisinde geçiyorsa bile ** Mükerrer No 0.0 'dır.

Bu yönergelere göre kodu hazıladıktan sonra bütün belgeler arasında Mükerrer No araması yapılıp sonuçlarını yazdırdık.

In [ ]:
satir = 2594
metin = df['data_text'][satir][:310].lower()
baslik = df['baslik'][satir].lower()
print("İçerik: \n",metin)
print("Başlık: \n",baslik)
print(metin.find("mükerrer"))
print(metin[266])

# ***ÇIKARIM:***
- **kanunum-nlp-doc-analysis-dataset.csv** dosyasındaki **4142** dosyanın arasında Mükerrer No Tespiti yapan kodumuz, **22** hata ile **0.99** oranında Doğruluk ile çalışmaktadır.
- Mükerrer No Tespiti yapan kodumuz; görev dahilinde olmadığı için **Komisyon Raporu, Genelge ve Özelge** kategorilerinde arama yapmamıştır.
- **(Önemli Bulgu)** Mükerrer No Tespiti yapan kodumuz; **Resmi Gazete, Kanun, Kanun Hükmünde Kararname ve Tüzük** kategorilerinde **Hatasız** çalışmaktadır.
- Mükerrer No Tespiti yapan kodumuz, **Yönetmelik** kategorisinde **1** hata ile **İstisnai Hatalı** çalışmaktadır.
- Mükerrer No Tespiti yapan kodumuz, **Tebliğ** kategorisinde **21** hata ile **İstisnai Hatalı/Hatalı** çalışmaktadır.


In [ ]:
# Kategorilere göre sınıflandırılmış Hata ve Doğruluk verilerini hesaplamak için gerekli değişkenleri oluşturuyoruz.
kategorilerimiz = df["kategori"].value_counts().index.values.tolist()
kategoridekiToplamBelgeSayisi = df["kategori"].value_counts().tolist()
kategoridekiHataliBelgeSayisi = [0] * len(kategorilerimiz)

# Kodda kullanılacak değişkenlerin oluşturulması
mukerrer = 0.0
baslangic_mukerrer = 0

for i in range(4142): # Sadece görev tanımında "mukerrer_no" değeri istenen kategorilerde arama yapıyoruz.
  if df['kategori'][i] in ["Komisyon Raporu", "Genelge", "Özelge"]:
    continue
  # VERİ TEMİZLEME İŞLEMLERİ
  baslik = df['baslik'][i]
  metin = df['data_text'][i][:310].lower()

  # Daha önceki çıkarımda belirtilen durumlara göre Mükerrer No araması yapıyoruz.
  if metin.find(". mükerrer") != -1:
      baslangic_mukerrer = metin.find(". mükerrer")
      mukerrer = float(metin[baslangic_mukerrer - 1])

  elif metin.find("mükerrer") != -1:
      baslangic_mukerrer = metin.find("mükerrer")
      if metin[baslangic_mukerrer+9] in '0123456789':
        mukerrer = 0.0
      else:
        mukerrer = 1.0
  else: 
    mukerrer = 0.0

  # df dosyasından 'mukerrer_no' değişkenini çekip uygun formata getiriyoruz.
  mukerrer_df = float(df['mukerrer_no'][i])

  if math.isnan(mukerrer_df): # Eğer çekilen değer boş idiyse döndürülen nan için işlem.
    mukerrer_df = 0.0

  # Bizim elde ettiğimiz verilerin asıl verilerle karşılaştırılması ve hatalar ile doğruluk oranlarının kategorilerce sayılması.
  if mukerrer != mukerrer_df:
      kategoridekiHataliBelgeSayisi[kategorilerimiz.index(df["kategori"][i])] += 1
for i in range(len(kategorilerimiz)):
  print(kategorilerimiz[i],':',kategoridekiHataliBelgeSayisi[i],'/',kategoridekiToplamBelgeSayisi[i],'=',round((kategoridekiToplamBelgeSayisi[i]-kategoridekiHataliBelgeSayisi[i])/kategoridekiToplamBelgeSayisi[i],3))
print('Toplam Hata:',sum(kategoridekiHataliBelgeSayisi))
print("Başarı oranı: ",(4142-sum(kategoridekiHataliBelgeSayisi))/4142)


# **Elde Edilen Bulguların Dokümantasyonu**

|Kategori|Toplam Belge|Doğru|Yanlış|Doğruluk Oranı|
|---|---|---|---|---|
|Kanun|514|514|0|1.0|
|Kanun Hükmünde Kararname|91|91|0|1.0|
|Resmi Gazete|546|546|0|1.0|
|Komisyon Raporu|500|500|0|1.0|
|Genelge|385|385|0|1.0|
|Cumhurbaşkanlığı Kararnamesi|54|54|0|1.0|
|Tüzük|82|82|0|1.0|
|Yönetmelik|597|596|1|0.998|
|Tebliğ|623|602|21|0.966|
|Özelge|750|750|0|1.0|
